In [8]:
# Import necessary libraries
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import libraries as lib

# Use lib.DataUitzoekenGui to load the data interactively
# Adjust the directory paths as needed
directory1 = "../../../OneDrive - HvA/Logfiles/SOLAR/2024/20240707_20240715_terugvaart/20240713_Huy-Limburg/0026 - 20240713T080342 14h37m16s 145.044km Durnal - Heel"
directory2 = "../../../OneDrive - HvA/Logfiles/SOLAR/2024/2024-10-24_sleeptest/0056 - 20241024T115530 1h25m5s 12.315km De Rijp - Uitgeest - De Rijp"

# Load dataset 1
print("Select files for Dataset 1:")
result1 = lib.DataUitzoekenGui(directory1)

# Load dataset 2
print("Select files for Dataset 2:")
result2 = lib.DataUitzoekenGui(directory2)

print("Both datasets are loaded successfully!")


Select files for Dataset 1:


SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '2_MPPT_0_05_05.csv', '3_MPPT_1_05_07.csv', '4_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

Select files for Dataset 2:


SelectMultiple(description='Bestanden:', options=('0_Bus.csv', '1_Master_08_05.csv', '2_MPPT_0_05_05.csv', '3_…

Button(button_style='success', description='Bestanden Laden', icon='check', style=ButtonStyle(), tooltip='Klik…

Label(value='')

Both datasets are loaded successfully!


In [9]:
print(result2)

{'merged_df':                  1_Datalogger port  1_Dataloggertijd, in s  1_Cycle count  \
Indextijd                                                                   
0 days 00:02:19                NaN                     NaN            NaN   
0 days 00:02:20                9.0                140.4098        145.861   
0 days 00:02:21                9.0                141.4102        146.860   
0 days 00:02:22                9.0                142.4364        147.886   
0 days 00:02:23                9.0                143.4079        148.859   
...                            ...                     ...            ...   
0 days 01:27:32                NaN                     NaN            NaN   
0 days 01:27:33                NaN                     NaN            NaN   
0 days 01:27:34                NaN                     NaN            NaN   
0 days 01:27:35                NaN                     NaN            NaN   
0 days 01:27:36                NaN                     NaN    